In [0]:
# Re-executar a criação do DataFrame (se você estiver em uma nova célula ou sessão)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

data = [
  ("Alice", 1, 1.70, "Engenharia"),
  ("Bob", 2, 1.85, "Ciência da Computação"),
  ("Charlie", 3, 1.75, "Engenharia"),
  ("David", 4, 1.90, "Matemática"),
  ("Eve", 5, 1.60, "Ciência da Computação"),
  ("Frank", 6, None, "Engenharia")
]

schema = StructType([
  StructField("Nome", StringType(), True),
  StructField("ID", IntegerType(), True),
  StructField("Altura", DoubleType(), True),
  StructField("Departamento", StringType(), True)
])

df = spark.createDataFrame(data, schema)

# Criar uma view temporária para usar com Spark SQL
df.createOrReplaceTempView("pessoas")

print("DataFrame original:")
df.show()
print("\nView temporária 'pessoas' criada.")

### a) Seleção e Renomeação de Colunas (Spark SQL):

In [0]:
%sql
-- Selecionar apenas algumas colunas
SELECT Nome, Departamento FROM pessoas;

-- Renomear uma coluna
SELECT Nome, ID, Altura, Departamento AS Area FROM pessoas;

-- Selecionar e renomear ao mesmo tempo
SELECT Nome AS Nome_Completo, ID, Altura AS Altura_Metros FROM pessoas;

### b) Filtragem de Dados (Spark SQL):

In [0]:
%sql
-- Filtrar por uma condição (pessoas da Engenharia)
SELECT * FROM pessoas WHERE Departamento = 'Engenharia';

-- Filtrar por múltiplas condições (ID > 3 E Altura > 1.70)
SELECT * FROM pessoas WHERE ID > 3 AND Altura > 1.70;

-- Filtrar por Altura > 1.80
SELECT * FROM pessoas WHERE Altura > 1.80;

### c) Criação e Modificação de Colunas (Spark SQL):

In [0]:
%sql
-- Adicionar uma nova coluna com valor constante
--SELECT *, 'Brasil' AS Pais FROM pessoas;

-- Criar uma nova coluna baseada em uma condição (ex: 'Status_Altura')
/*SELECT
*,
CASE
  WHEN Altura >= 1.80 THEN 'Alto'
  WHEN Altura < 1.70 THEN 'Baixo'
  ELSE 'Medio'
END AS Status_Altura
FROM pessoas;*/

-- Modificar uma coluna existente (ex: Altura em cm)
SELECT Nome, ID, Altura * 100 AS Altura_cm, Departamento FROM pessoas;

### d) Agregações (Spark SQL):

In [0]:
%sql
-- Contar o número de pessoas por departamento
--SELECT Departamento, COUNT(*) AS Contagem FROM pessoas GROUP BY Departamento;

-- Calcular a média da altura por departamento
--SELECT Departamento, AVG(Altura) AS Media_Altura FROM pessoas GROUP BY Departamento;

-- Múltiplas agregações
SELECT
Departamento,
AVG(Altura) AS Media_Altura,
MIN(Altura) AS Min_Altura,
MAX(Altura) AS Max_Altura
FROM pessoas
GROUP BY Departamento;

### e) Ordenação (Spark SQL):

In [0]:
%sql
-- Ordenar por Altura (crescente)
SELECT * FROM pessoas ORDER BY Altura ASC;

-- Ordenar por Departamento (crescente) e depois por Altura (decrescente)
SELECT * FROM pessoas ORDER BY Departamento ASC, Altura DESC;

### f) Remoção de Duplicatas (Spark SQL):

In [0]:
# Adicionar um dado duplicado para demonstração
data_duplicado = data + [("Alice", 1, 1.70, "Engenharia")]
df_com_duplicata = spark.createDataFrame(data_duplicado, schema)
df_com_duplicata.createOrReplaceTempView("pessoas_com_duplicata")

print("\nDataFrame com duplicata (para SQL):")
df_com_duplicata.show()

In [0]:
%sql
-- Remover linhas duplicadas (considera todas as colunas)
--SELECT DISTINCT * FROM pessoas_com_duplicata;

-- Remover duplicatas com base em um subconjunto de colunas (ex: Nome e Departamento)
-- Isso é um pouco mais complexo em SQL puro sem funções de janela, mas pode ser feito com GROUP BY
-- ou com funções de janela (ROW_NUMBER) para selecionar a primeira ocorrência.
-- Usando GROUP BY para simular a remoção de duplicatas em um subconjunto:
/*SELECT Nome, ID, Altura, Departamento
FROM (
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY Nome, Departamento ORDER BY ID) as rn
FROM pessoas_com_duplicata
)
WHERE rn = 1;*/

-- Ou, se você quer apenas as colunas Nome e Departamento sem duplicatas:
SELECT DISTINCT Nome, Departamento FROM pessoas_com_duplicata;

### g) Tratamento de Valores Nulos (Spark SQL):

In [0]:
%sql
-- DataFrame com valor nulo (Frank tem Altura nula)
-- A view 'pessoas' já tem o valor nulo para Frank.

-- Remover linhas com qualquer valor nulo
--SELECT * FROM pessoas WHERE Altura IS NOT NULL AND Nome IS NOT NULL AND ID IS NOT NULL AND Departamento IS NOT NULL;
-- Ou, mais conciso se você sabe quais colunas podem ter nulos:
--SELECT * FROM pessoas WHERE Altura IS NOT NULL;

-- Preencher valores nulos em uma coluna específica
/*SELECT
Nome,
ID,
COALESCE(Altura, 0.0) AS Altura, -- Substitui NULL por 0.0 na coluna Altura
Departamento
FROM pessoas;*/

-- Preencher valores nulos em todas as colunas do mesmo tipo (ex: Departamento)
SELECT
Nome,
ID,
Altura,
COALESCE(Departamento, 'Desconhecido') AS Departamento
FROM pessoas;


In [0]:
# Exemplo de como executar SQL e obter um DataFrame PySpark
df_engenharia_sql = spark.sql("SELECT * FROM pessoas WHERE Departamento = 'Engenharia'")
print("\nResultado da query SQL em um DataFrame PySpark:")
df_engenharia_sql.show()

In [0]:
# Importar tipos de dados do PySpark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# DataFrame de Pessoas (usando os dados anteriores)
data_pessoas = [
("Alice", 1, 1.70, "Engenharia"),
("Bob", 2, 1.85, "Ciência da Computação"),
("Charlie", 3, 1.75, "Engenharia"),
("David", 4, 1.90, "Matemática"),
("Eve", 5, 1.60, "Ciência da Computação"),
("Frank", 6, None, "Engenharia"),
("Grace", 7, 1.68, "Marketing") # Adicionando um novo para demonstração de joins
]

schema_pessoas = StructType([
StructField("Nome", StringType(), True),
StructField("ID", IntegerType(), True),
StructField("Altura", DoubleType(), True),
StructField("Departamento", StringType(), True)
])

df_pessoas = spark.createDataFrame(data_pessoas, schema_pessoas)
df_pessoas.createOrReplaceTempView("pessoas") # Criar view temporária para SQL

print("DataFrame de Pessoas:")
df_pessoas.show()

# DataFrame de Departamentos
data_departamentos = [
("Engenharia", "ENG", "São Paulo"),
("Ciência da Computação", "COMP", "Rio de Janeiro"),
("Matemática", "MAT", "Belo Horizonte"),
("Recursos Humanos", "RH", "São Paulo") # Departamento que não tem pessoas ainda
]

schema_departamentos = StructType([
StructField("Nome_Departamento", StringType(), True),
StructField("Sigla_Departamento", StringType(), True),
StructField("Localizacao", StringType(), True)
])

df_departamentos = spark.createDataFrame(data_departamentos, schema_departamentos)
df_departamentos.createOrReplaceTempView("departamentos") # Criar view temporária para SQL

print("\nDataFrame de Departamentos:")
df_departamentos.show()

### a) INNER JOIN (Junção Interna)

O que faz: Retorna apenas as linhas onde há correspondência em ambos os DataFrames/tabelas.
Quando usar: Quando você só quer os registros que têm informações em todas as fontes.

In [0]:
# INNER JOIN
df_inner_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "inner")
print("\nINNER JOIN:")
df_inner_join.show()

In [0]:
%sql
-- INNER JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
INNER JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

###b) LEFT (OUTER) JOIN (Junção Externa Esquerda)

O que faz: Retorna todas as linhas do DataFrame/tabela da esquerda e as linhas correspondentes do DataFrame/tabela da direita. Se não houver correspondência na direita, os valores serão null.
Quando usar: Quando você quer manter todos os registros da tabela principal (esquerda) e adicionar informações da tabela secundária (direita, se houver).

In [0]:
# LEFT JOIN
df_left_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "left_outer")
print("\nLEFT JOIN:")
df_left_join.show()

In [0]:
%sql
-- LEFT JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
LEFT JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

### c) RIGHT (OUTER) JOIN (Junção Externa Direita)

O que faz: Retorna todas as linhas do DataFrame/tabela da direita e as linhas correspondentes do DataFrame/tabela da esquerda. Se não houver correspondência na esquerda, os valores serão null.
Quando usar: Quando você quer manter todos os registros da tabela secundária (direita) e adicionar informações da tabela principal (esquerda, se houver).

In [0]:
# RIGHT JOIN
df_right_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "right_outer")
print("\nRIGHT JOIN:")
df_right_join.show()

In [0]:
%sql
-- RIGHT JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
RIGHT JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

### d) FULL (OUTER) JOIN (Junção Externa Completa)

O que faz: Retorna todas as linhas de ambos os DataFrames/tabelas. Se não houver correspondência em um dos lados, os valores serão null para as colunas desse lado.
Quando usar: Quando você quer ver todos os registros de ambas as tabelas, independentemente de haver correspondência.

In [0]:
# FULL JOIN
df_full_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "full_outer")
print("\nFULL JOIN:")
df_full_join.show()

In [0]:
%sql
-- FULL JOIN
SELECT p.Nome, p.Departamento, d.Sigla_Departamento, d.Localizacao
FROM pessoas p
FULL JOIN departamentos d ON p.Departamento = d.Nome_Departamento;

### e) ANTI JOIN (Junção Anti)

O que faz: Retorna as linhas do DataFrame/tabela da esquerda que não têm correspondência no DataFrame/tabela da direita. É o oposto de um INNER JOIN para o lado esquerdo.
Quando usar: Para encontrar registros "órfãos" ou identificar dados que não existem em outra tabela.

In [0]:
# ANTI JOIN
df_anti_join = df_pessoas.join(df_departamentos, df_pessoas.Departamento == df_departamentos.Nome_Departamento, "left_anti")
print("\nANTI JOIN (Pessoas sem Departamento correspondente na tabela de Departamentos):")
df_anti_join.show()

In [0]:
%sql
-- ANTI JOIN (usando NOT EXISTS ou LEFT JOIN com WHERE IS NULL)
-- Opção 1: Usando NOT EXISTS
SELECT p.Nome, p.Departamento
FROM pessoas p
WHERE NOT EXISTS (
SELECT 1 FROM departamentos d WHERE p.Departamento = d.Nome_Departamento
);

-- Opção 2: Usando LEFT JOIN com WHERE IS NULL (comum para simular ANTI JOIN)
SELECT p.Nome, p.Departamento
FROM pessoas p
LEFT JOIN departamentos d ON p.Departamento = d.Nome_Departamento
WHERE d.Nome_Departamento IS NULL;

### Tópico: Funções de Janela (Window Functions)

O que são Funções de Janela?

Ao contrário das funções de agregação (SUM, AVG, COUNT) que colapsam linhas em um único resultado por grupo (GROUP BY), as funções de janela retornam um valor para cada linha do DataFrame, mas esse valor é calculado com base em um "grupo" de linhas relacionadas (a janela).

**Componentes** Chave de uma Função de Janela:

Função de Janela: A função de agregação ou analítica a ser aplicada (ex: ROW_NUMBER(), RANK(), LAG(), SUM(), AVG()).
`OVER() Clause`: Define a janela sobre a qual a função será aplicada. Ela tem três partes principais:
`PARTITION BY`: Divide os dados em partições (grupos) independentes. A função de janela é aplicada separadamente dentro de cada partição. (Similar ao GROUP BY, mas não colapsa as linhas).
`ORDER BY`: Define a ordem das linhas dentro de cada partição. Isso é crucial para funções que dependem da ordem, como ranking, LAG, LEAD, ou somas cumulativas.
`ROWS BETWEEN / RANGE BETWEEN`: Define os limites da janela dentro de cada partição. Por exemplo, "as 3 linhas anteriores e a linha atual" (ROWS BETWEEN 3 PRECEDING AND CURRENT ROW). Se omitido, geralmente a janela padrão é do início da partição até a linha atual (RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) ou toda a partição (ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING), dependendo da função e do contexto.

In [0]:
# Re-executar a criação do DataFrame de Pessoas (se você estiver em uma nova célula ou sessão)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import lit

data_pessoas = [
  ("Alice", 1, 1.70, "Engenharia", 7000),
  ("Bob", 2, 1.85, "Ciência da Computação", 8500),
  ("Charlie", 3, 1.75, "Engenharia", 7500),
  ("David", 4, 1.90, "Matemática", 9000),
  ("Eve", 5, 1.60, "Ciência da Computação", 8000),
  ("Frank", 6, None, "Engenharia", 6000),
  ("Grace", 7, 1.68, "Marketing", 6500),
  ("Heidi", 8, 1.72, "Engenharia", 7000) # Adicionando mais um para ranking
]

schema_pessoas = StructType([
  StructField("Nome", StringType(), True),
  StructField("ID", IntegerType(), True),
  StructField("Altura", DoubleType(), True),
  StructField("Departamento", StringType(), True),
  StructField("Salario", IntegerType(), True)
])

df_pessoas_salario = spark.createDataFrame(data_pessoas, schema_pessoas)
df_pessoas_salario.createOrReplaceTempView("pessoas_salario") # Criar view temporária para SQL

print("DataFrame de Pessoas com Salário:")
df_pessoas_salario.show()

### 3. Exemplos de Funções de Janela

a) Funções de Ranking (ROW_NUMBER, RANK, DENSE_RANK, NTILE)

ROW_NUMBER(): Atribui um número sequencial único a cada linha dentro de sua partição, começando em 1.
RANK(): Atribui um rank a cada linha dentro de sua partição. Se houver empates, as linhas empatadas recebem o mesmo rank, e o próximo rank é "pulado".
DENSE_RANK(): Similar ao RANK(), mas não pula ranks em caso de empates.
NTILE(n): Divide as linhas em n grupos (tiles) de tamanho aproximadamente igual e atribui um número de grupo a cada linha.

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, ntile, col

# Definir a especificação da janela: Particionar por Departamento, ordenar por Salario (decrescente)
window_spec = Window.partitionBy("Departamento").orderBy(col("Salario").desc())

# Aplicar funções de ranking
df_ranking = df_pessoas_salario.withColumn("row_num", row_number().over(window_spec)) \
                             .withColumn("rank", rank().over(window_spec)) \
                             .withColumn("dense_rank", dense_rank().over(window_spec)) \
                             .withColumn("ntile_2", ntile(2).over(window_spec)) # Dividir em 2 grupos

print("\nFunções de Ranking por Departamento (Salário Decrescente):")
df_ranking.show()

In [0]:
%sql
-- Funções de Ranking por Departamento (Salário Decrescente)
SELECT
Nome,
Departamento,
Salario,
ROW_NUMBER() OVER (PARTITION BY Departamento ORDER BY Salario DESC) AS row_num,
RANK() OVER (PARTITION BY Departamento ORDER BY Salario DESC) AS rank,
DENSE_RANK() OVER (PARTITION BY Departamento ORDER BY Salario DESC) AS dense_rank,
NTILE(2) OVER (PARTITION BY Departamento ORDER BY Salario DESC) AS ntile_2
FROM pessoas_salario;

### b) Funções de Deslocamento (LAG, LEAD)

LAG(coluna, offset, default): Retorna o valor de uma coluna de uma linha anterior dentro da mesma partição.
LEAD(coluna, offset, default): Retorna o valor de uma coluna de uma linha posterior dentro da mesma partição.

In [0]:
from pyspark.sql.functions import lag, lead

# Definir a especificação da janela: Particionar por Departamento, ordenar por Salario (crescente)
window_spec_order_salario = Window.partitionBy("Departamento").orderBy("Salario")

# Aplicar funções LAG e LEAD
df_lag_lead = df_pessoas_salario.withColumn("salario_anterior", lag("Salario", 1).over(window_spec_order_salario)) \
                              .withColumn("salario_proximo", lead("Salario", 1).over(window_spec_order_salario))

print("\nFunções LAG e LEAD (Salário Anterior/Próximo por Departamento):")
df_lag_lead.show()

In [0]:
%sql
-- Funções LAG e LEAD (Salário Anterior/Próximo por Departamento)
SELECT
Nome,
Departamento,
Salario,
LAG(Salario, 1) OVER (PARTITION BY Departamento ORDER BY Salario) AS salario_anterior,
LEAD(Salario, 1) OVER (PARTITION BY Departamento ORDER BY Salario) AS salario_proximo
FROM pessoas_salario;

### c) Agregações de Janela (Soma Cumulativa, Média Móvel)

Você pode usar funções de agregação como SUM, AVG, COUNT, MIN, MAX como funções de janela, definindo os limites da janela com ROWS BETWEEN ou RANGE BETWEEN.

In [0]:
from pyspark.sql.functions import sum, avg

# Definir a especificação da janela para soma cumulativa: Particionar por Departamento, ordenar por Salario
# A janela vai do início da partição até a linha atual
window_spec_cumulative = Window.partitionBy("Departamento").orderBy("Salario").rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Definir a especificação da janela para média móvel: Particionar por Departamento, ordenar por Salario
# A janela vai da linha anterior até a linha atual
window_spec_moving_avg = Window.partitionBy("Departamento").orderBy("Salario").rowsBetween(-1, Window.currentRow)

# Aplicar soma cumulativa e média móvel
df_cumulative_moving = df_pessoas_salario.withColumn("salario_acumulado", sum("Salario").over(window_spec_cumulative)) \
                                       .withColumn("media_movel_salario", avg("Salario").over(window_spec_moving_avg))

print("\nSoma Cumulativa e Média Móvel de Salário por Departamento:")
df_cumulative_moving.show()

In [0]:
%sql
-- Soma Cumulativa e Média Móvel de Salário por Departamento
SELECT
Nome,
Departamento,
Salario,
SUM(Salario) OVER (PARTITION BY Departamento ORDER BY Salario ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS salario_acumulado,
AVG(Salario) OVER (PARTITION BY Departamento ORDER BY Salario ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS media_movel_salario
FROM pessoas_salario;

### Tópico: Trabalhando com Tipos de Dados Complexos (Arrays, Structs, Maps)

No mundo real, os dados raramente são "planos" (**flat**). É muito comum encontrar dados aninhados ou semi-estruturados, especialmente ao lidar com fontes como **JSON**, **XML** ou **logs**. O Spark, através de seus DataFrames, oferece suporte robusto para trabalhar com esses tipos de dados complexos: `Structs`, `Arrays` e `Maps`.

Compreender como manipular esses tipos é crucial para extrair informações valiosas e transformar dados para análise.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType
from pyspark.sql.functions import col, explode, posexplode, map_keys, map_values, lit, array_contains, struct, create_map

# Dados de exemplo com tipos complexos
data_complexos = [
  ("Alice", 1, {"rua": "Rua A", "numero": 10}, ["Python", "SQL"], {"projeto1": "ativo", "projeto2": "concluido"}),
  ("Bob", 2, {"rua": "Av. B", "numero": 25}, ["Java", "Spark", "SQL"], {"projeto3": "ativo"}),
  ("Charlie", 3, {"rua": "Rua C", "numero": 5}, ["Python", "Scala"], {}),
  ("David", 4, None, ["C++"], {"projeto4": "pendente", "projeto5": "ativo"}), # Exemplo com struct nulo
  ("Eve", 5, {"rua": "Trav. D", "numero": 12}, [], {"projeto6": "concluido"}) # Exemplo com array vazio
]

# Definir o esquema (schema) do DataFrame
schema_complexos = StructType([
  StructField("Nome", StringType(), True),
  StructField("ID", IntegerType(), True),
  StructField("Endereco", StructType([
      StructField("rua", StringType(), True),
      StructField("numero", IntegerType(), True)
  ]), True),
  StructField("Habilidades", ArrayType(StringType()), True),
  StructField("StatusProjetos", MapType(StringType(), StringType()), True)
])

df_complexos = spark.createDataFrame(data_complexos, schema_complexos)
df_complexos.createOrReplaceTempView("dados_complexos") # Criar view temporária para SQL

print("DataFrame com Tipos Complexos:")
df_complexos.printSchema()
df_complexos.show(truncate=False)

### 2. Manipulando Structs (Registros Aninhados)

### a) Acessando Campos de um Struct:

In [0]:
# Acessar campos específicos do struct 'Endereco'
df_struct_acesso = df_complexos.select("Nome", "Endereco.rua", col("Endereco.numero").alias("num_casa"))
print("\nAcessando campos de Struct:")
df_struct_acesso.show(truncate=False)

In [0]:
%sql
-- Acessar campos específicos do struct 'Endereco'
SELECT Nome, Endereco.rua, Endereco.numero AS num_casa
FROM dados_complexos;

### b) Criando um Novo Struct:

In [0]:
# Criar um novo struct a partir de colunas existentes
df_novo_struct = df_complexos.withColumn("InfoPessoal", struct(col("Nome").alias("nome_completo"), col("ID").alias("identificador")))
print("\nCriando um novo Struct:")
df_novo_struct.printSchema()
df_novo_struct.show(truncate=False)

In [0]:
%sql
-- Criar um novo struct a partir de colunas existentes
SELECT
Nome,
ID,
Endereco,
Habilidades,
StatusProjetos,
STRUCT(Nome AS nome_completo, ID AS identificador) AS InfoPessoal
FROM dados_complexos;

### 3. Manipulando Arrays (Listas de Elementos)
Um ArrayType é uma lista de elementos do mesmo tipo.

### a) Explodindo Arrays (explode, posexplode):

- explode: Transforma cada elemento de um array em uma nova linha. Se o array for nulo ou vazio, a linha original é descartada.

- posexplode: Similar ao explode, mas também adiciona uma coluna com a posição (índice) do elemento no array.

In [0]:
# Explodir o array 'Habilidades'
df_explode = df_complexos.select("Nome", explode("Habilidades").alias("Habilidade_Individual"))
print("\nExplodindo Array (explode):")
df_explode.show(truncate=False)

# Explodir o array 'Habilidades' com posição
df_posexplode = df_complexos.select("Nome", posexplode("Habilidades").alias("posicao", "Habilidade_Individual"))
print("\nExplodindo Array com Posição (posexplode):")
df_posexplode.show(truncate=False)

In [0]:
%sql
-- Explodir o array 'Habilidades'
SELECT Nome, Habilidade_Individual
FROM dados_complexos
LATERAL VIEW explode(Habilidades) AS Habilidade_Individual;

-- Explodir o array 'Habilidades' com posição
SELECT Nome, posicao, Habilidade_Individual
FROM dados_complexos
LATERAL VIEW posexplode(Habilidades) AS posicao, Habilidade_Individual;

### b) Filtrando por Conteúdo do Array (array_contains):

In [0]:
# Filtrar pessoas que possuem a habilidade 'Python'
df_python_skills = df_complexos.filter(array_contains(col("Habilidades"), "Python"))
print("\nPessoas com habilidade 'Python':")
df_python_skills.show(truncate=False)

In [0]:
%sql
-- Filtrar pessoas que possuem a habilidade 'Python'
SELECT *
FROM dados_complexos
WHERE array_contains(Habilidades, 'Python');

### 4. Manipulando Maps (Pares Chave-Valor)

Um MapType é uma coleção de pares chave-valor.

### a) Acessando Valores por Chave:

In [0]:
# Acessar o status de um projeto específico (ex: 'projeto1')
df_map_acesso = df_complexos.select("Nome", col("StatusProjetos")["projeto1"].alias("Status_Projeto1"))
print("\nAcessando valor de Map por chave:")
df_map_acesso.show(truncate=False)

In [0]:
%sql
-- Acessar o status de um projeto específico (ex: 'projeto1')
SELECT Nome, StatusProjetos['projeto1'] AS Status_Projeto1
FROM dados_complexos;

### b) Obtendo Chaves e Valores do Map:

In [0]:
# Obter todas as chaves e valores do map
df_map_keys_values = df_complexos.select("Nome", map_keys("StatusProjetos").alias("Chaves_Projetos"), map_values("StatusProjetos").alias("Valores_Projetos"))
print("\nObtendo Chaves e Valores de Map:")
df_map_keys_values.show(truncate=False)

In [0]:
%sql
-- Obter todas as chaves e valores do map
SELECT Nome, map_keys(StatusProjetos) AS Chaves_Projetos, map_values(StatusProjetos) AS Valores_Projetos
FROM dados_complexos;

### c) Explodindo Maps (explode):

Assim como arrays, maps também podem ser "explodidos" para transformar cada par chave-valor em uma nova linha.

In [0]:
# Explodir o map 'StatusProjetos'
df_explode_map = df_complexos.select("Nome", explode("StatusProjetos").alias("Projeto", "Status"))
print("\nExplodindo Map (explode):")
df_explode_map.show(truncate=False)

In [0]:
%sql
-- Explodir o map 'StatusProjetos'
SELECT Nome, Projeto, Status
FROM dados_complexos
LATERAL VIEW explode(StatusProjetos) AS Projeto, Status;